# Average Daily Trading Volume & Intraday Volume Spikes Vs Rolling Mean (Completed)

In [157]:
# Import relevant libraries

import numpy as np
import pandas as pd

from pandas import Series, DataFrame

In [158]:
# Read CSV
# Clean 1-minute OHLCV dataset (2 trading days, 09:30 - 16:00)

df = pd.read_csv('ohlcv_demo_1min.csv',parse_dates = ['datetime'])

df.head()

,datetime,symbol,open,high,low,close,volume
0,2025-01-06 09:30:00,DEMO,100.03,100.08,99.97,100.02,894
1,2025-01-06 09:31:00,DEMO,100.03,100.04,100.01,100.02,844
2,2025-01-06 09:32:00,DEMO,100.06,100.14,99.97,100.05,887
3,2025-01-06 09:33:00,DEMO,100.15,100.21,100.07,100.13,779
4,2025-01-06 09:34:00,DEMO,100.13,100.16,100.08,100.11,911


In [159]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 782 entries, 0 to 781
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   datetime  782 non-null    datetime64[ns]
 1   symbol    782 non-null    object        
 2   open      782 non-null    float64       
 3   high      782 non-null    float64       
 4   low       782 non-null    float64       
 5   close     782 non-null    float64       
 6   volume    782 non-null    int64         
dtypes: datetime64[ns](1), float64(4), int64(1), object(1)
memory usage: 42.9+ KB


In [160]:
# Check for missing or NaN values

df.isna().sum()

datetime    0
symbol      0
open        0
high        0
low         0
close       0
volume      0
dtype: int64

In [161]:
# set_index as datetime, followed by sort_index

df = df.set_index('datetime').sort_index()

df

,symbol,open,high,low,close,volume
datetime,,,,,,
2025-01-06 09:30:00,DEMO,100.03,100.08,99.97,100.02,894
2025-01-06 09:31:00,DEMO,100.03,100.04,100.01,100.02,844
2025-01-06 09:32:00,DEMO,100.06,100.14,99.97,100.05,887
2025-01-06 09:33:00,DEMO,100.15,100.21,100.07,100.13,779
2025-01-06 09:34:00,DEMO,100.13,100.16,100.08,100.11,911
...,...,...,...,...,...,...
2025-01-07 15:56:00,DEMO,100.97,101.00,100.93,100.96,829
2025-01-07 15:57:00,DEMO,100.95,101.04,100.90,100.99,927
2025-01-07 15:58:00,DEMO,100.97,101.02,100.93,100.98,869


## Q1. Average daily trading volume

#### Using the minute bars, compute the average daily volume across the dataset (i.e., take the total volume for each day, then average these daily totals).

 
 


Average Daily Trading Volume (ADTV) refers to the average number of shares or contracts traded in a specific security daily over a given time frame, such as 20 or 30 days. You can compare the current trading volume with the ADTV to determine whether the volume is particularly high or low. 


The primary purpose of ADTV is to indicate how actively a security is being traded. It helps traders and investors gauge:
- Liquidity: Whether a security can be bought or sold quickly
- Market interest - The level of demand for the security among investors
- Price stability - The likelihood of significant price swings due to trading activity 

Securities with high ADTV are likely to have stable prices, tighter bid-ask spreads, and less slippage (the difference between the expected price and the actual execution price). 

Conversely, securities with low ADTV may face higher transaction costs, larger bid-ask spreads, and increased price volatility.  
  
Calculation:
To find the ADTV, you calculate the total trading volume over a specific period and divide it by the number of trading days within that period. 




In [162]:
# Aggregate intraday volume

df_1d = df.resample('1d').agg({
    'open': 'first', 'high': 'max', 'low': 'min', 'close': 'last', 'volume': 'sum'})

df_1d

,open,high,low,close,volume
datetime,,,,,
2025-01-06,100.03,100.60,99.26,100.33,366920
2025-01-07,100.01,101.04,99.73,100.89,363557


In [163]:
# Compute the 2-day average daily trading volume (ADTV)

ADTV_2 = df_1d['volume'].mean()

ADTV_2


365238.5

## Q2. Intraday volume spikes vs rolling mean
#### Using a 15-minute rolling mean of volume (within each day), find all minutes where volume > 2 × rolling_mean_15m.  
#### List the spike timestamps and how many spikes occurred per day.



In [164]:
# If computed this way, 15-minute rolling mean of volume is not within each day

df['vol_ma_15'] = df['volume'].rolling(15).mean()

df


# 15-minute rolling mean of volume (within each day)
# Computation should reset at the day boundary so the window never reaches into the previous/next day 
# Be careful of index misalignment (multi-index output), implement reset_index to prevent this 

df['VOL_MA_15'] = df.groupby(df.index.floor('D'))['volume'].rolling('15min', min_periods = 15).mean().reset_index(level = 0, drop = True)

df


,symbol,open,high,low,close,volume,vol_ma_15,VOL_MA_15
datetime,,,,,,,,
2025-01-06 09:30:00,DEMO,100.03,100.08,99.97,100.02,894,NaN,NaN
2025-01-06 09:31:00,DEMO,100.03,100.04,100.01,100.02,844,NaN,NaN
2025-01-06 09:32:00,DEMO,100.06,100.14,99.97,100.05,887,NaN,NaN
2025-01-06 09:33:00,DEMO,100.15,100.21,100.07,100.13,779,NaN,NaN
2025-01-06 09:34:00,DEMO,100.13,100.16,100.08,100.11,911,NaN,NaN
...,...,...,...,...,...,...,...,...
2025-01-07 15:56:00,DEMO,100.97,101.00,100.93,100.96,829,852.000000,852.000000
2025-01-07 15:57:00,DEMO,100.95,101.04,100.90,100.99,927,860.733333,860.733333
2025-01-07 15:58:00,DEMO,100.97,101.02,100.93,100.98,869,863.333333,863.333333


In [165]:
# Find all datetime where there is a spike
# IsSpike is True if volume >= 2 x rolling mean

df['volume ratio'] = df['volume'] / df['VOL_MA_15']

df['IsSpike'] = df['volume ratio'] >= 2

df.loc [ (df['IsSpike'] == True) ]


,symbol,open,high,low,close,volume,vol_ma_15,VOL_MA_15,volume ratio,IsSpike
datetime,,,,,,,,,,
2025-01-06 10:00:00,DEMO,99.72,99.80,99.61,99.69,6492,1362.533333,1362.533333,4.764654,True
2025-01-06 12:00:00,DEMO,99.37,99.42,99.35,99.40,5685,1429.200000,1429.200000,3.977750,True
2025-01-06 15:30:00,DEMO,100.06,100.07,100.05,100.06,4488,999.866667,999.866667,4.488598,True
2025-01-07 10:00:00,DEMO,100.10,100.14,100.06,100.10,5994,1333.933333,1333.933333,4.493478,True
2025-01-07 12:00:00,DEMO,100.18,100.24,100.14,100.20,5750,1425.533333,1425.533333,4.033578,True
2025-01-07 15:30:00,DEMO,100.61,100.65,100.58,100.62,3805,919.066667,919.066667,4.140070,True


In [166]:
# Number of spikes occurred per day

spike_per_day = df.groupby(df.index.floor('D'))['IsSpike'].sum()    # Code can be formatted this way as our index is already datetime

spike_per_day



datetime
2025-01-06    3
2025-01-07    3
Name: IsSpike, dtype: int64